https://mp.weixin.qq.com/s?__biz=MzUyMzg0ODY0Ng==&mid=2247483839&idx=1&sn=e3a1e7753a283f9dcc8ecdf5354230eb&chksm=fa371f16cd409600b2de38a95376bd1f1d26919a67a804b30c53dfc96a066763ba2036315440&mpshare=1&scene=1&srcid=0330XnX8Q7x1BsPQCGHecsFv&sharer_sharetime=1585548755034&sharer_shareid=d88df2ec93a2e5e478eaa39ef5a82e2f&key=fa0f4cfef200c085620487de7d82882718619ab9893ba6f4b3100366149d2e86a8943803d07a5e6012ad4f5d6c64ccdf38ecc4d80ef93dc6f5a8742dff9388bba18d9b211bdd1a94a05f2fd7d897de7a&ascene=1&uin=MjA1MjAyODkxNg%3D%3D&devicetype=Windows+10&version=62080079&lang=zh_CN&exportkey=Ab9j2hoNaEcCHzzhs101Hxo%3D&pass_ticket=oQV%2B4QgocE%2B79igKS84ByQiBvNr7zSd0fGMluYqPBYLpNaLxouEPfg16iqZpY1vp

In [1]:
# 导包
import math
import re
import numpy as np
import tensorflow as tf
from collections import Counter

In [ ]:
# 数据预处理
# 数据路径
DATA_PATH = '../dataset/poetry.txt'
# 单行诗最大长度
MAX_LEN = 64
# 禁用的字符，拥有以下字符号的诗将被忽略
